In [1]:
# !pip install punctuators
# !pip install flair
# !pip install spacy
# !python -m spacy download en_core_web_sm

In [2]:
import torch
from torch import nn
import json
from typing import List
from tqdm import tqdm
import string
import spacy
import re
# from punctuators.models import SBDModelONNX
from augment import AugmentData

# TEST

In [3]:
ag = AugmentData()
# sentence_pool, span_sentence_pool = ag.augment("/media/s24gb1/90a7e21c-edf4-4782-a0eb-731b73c521c2/DUC/NAACL2024/data/ECF 2.0/test/Subtask_1_test.json")

In [4]:
text = "Whites . Okay ."
# g.spacy(text)
# l, x = ag.augment_sentence(text)
# for i in range(len(l)):
#     print(l[i])
#     # if len(x[i])>2:
#     print(x[i])
ag.augment_sentence(text, test=True)
# pool = ag.split_comma(text)
# ag.concat_comma(pool)

[('NOUN', 'white'), ('PUNCT', '.'), ('INTJ', 'okay'), ('PUNCT', '.')]
spacy:['Whites .', 'Whites . Okay .', 'Whites . Okay . ', 'Okay .', 'Okay . ', 'Whites . Okay .', 'Whites .', 'Whites . Okay .', 'Okay .', 'Whites', 'Whites .', 'Whites . Okay .', 'Okay .']
remove_list_start_by_coordinating_conjunction:['Whites .', 'Whites . Okay .', 'Whites . Okay . ', 'Okay .', 'Okay . ', 'Whites . Okay .', 'Whites .', 'Whites . Okay .', 'Okay .', 'Whites', 'Whites .', 'Whites . Okay .', 'Okay .']
remove_duplicate:['Whites .', 'Whites . Okay .', 'Whites . Okay . ', 'Okay .', 'Okay . ', 'Whites']
remove_unclear_content:['Whites .']
remove_end_by_unclear_sentence:['Whites .']
remove_dup_words:['Whites .']
remove_dup_sentences:['Whites .']
remove_unclear_last_sentence:['Whites .']
remove_unclear_first_sentence:['Whites .']
remove_interjections_in_the_end:['Whites .']
remove_more_adj:['Whites .']
remove_more_intj:['Whites .']


['Whites .']

In [5]:
# text = "Hey ! I did ! But the store would not take her back ! So then I took her to the shelter , and you know what I found out ? If they can not find a home for her , they kill her ! And I am not gonna let that happen to little Yasmine ! Okay , good , good , good , cause , good , cause I was kinda having second thoughts too . Okay . And it is not just chicks you know ? It is all kinds of other animals !"
# g.spacy(text)
# l, x = ag.augment_sentence(text)
# for i in range(len(l)):
#     print(l[i])
#     # if len(x[i])>2:
#     print(x[i])
# s, u = ag.augment_sentence(text, test=True)

In [6]:
# import spacy

# # Load the English language model
# nlp = spacy.load("en_core_web_sm")

# # Input sentence
# sentence = "You know you probably did not know this , but back in high school , I had a , um , major crush on you ."

# # Process the sentence with spaCy
# doc = nlp(sentence)

# # Split the sentence by pronouns
# split_sentences = []
# current_sentence = []

# for token in doc:
#     # Check if the token is a pronoun
#     if token.pos_ == 'PRON':
#         split_sentences.append(' '.join(current_sentence))
#         current_sentence = [token.text]
#     else:
#         current_sentence.append(token.text)

# # Add the remaining part of the sentence if any
# if current_sentence:
#     split_sentences.append(' '.join(current_sentence))

# # Print the split sentences
# for i, split_sentence in enumerate(split_sentences):
#     print(f"Sentence {i + 1}: {split_sentence}")


In [7]:
!cd GraphEmbedding/
!python setup.py install

/bin/bash: line 0: cd: GraphEmbedding/: No such file or directory
python: can't open file 'setup.py': [Errno 2] No such file or directory


In [8]:
# text = "Well , you might try accidentally breaking something valuable of hers , say her"
# ag.spacy(text)

In [9]:
# punctuation_chars = string.punctuation

#         # Remove punctuation at the beginning and end of the string
# cleaned_text = text.strip(punctuation_chars)
# cleaned_text